In [1]:
# import packages
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings("ignore")

<h3>Q1. Downloading the data</h3>

In [2]:
# get data
jan_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet')
feb_data = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')

In [3]:
# head of dataframe
jan_data.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [4]:
# columns are in January data
print(f"There are {jan_data.shape[1]} columns in January data")

There are 19 columns in January data


<h3>Q2. Computing duration</h3>

In [5]:
# standard deviation of january trip
jan_data['duration'] = jan_data.tpep_dropoff_datetime - jan_data.tpep_pickup_datetime
jan_data.duration = jan_data.duration.apply(lambda td: td.total_seconds() / 60) # ride duration in minutes
print("The standard deviation of the trips in January was {:.2f}".format(jan_data.duration.std()))

The standard deviation of the trips in January was 42.59


<h3>Q3. Dropping outliers</h3>

In [6]:
# fraction of record left after outlier removal
clean_jan_data = jan_data[(jan_data.duration >= 1) & (jan_data.duration <= 60)]
fraction_left = clean_jan_data.shape[0] / jan_data.shape[0]
print("Fraction left is {:.0%}".format(fraction_left))

Fraction left is 98%


<h3>Q4. One-hot encoding</h3>

In [7]:
# apply one-hot encoding
categorical = ['PULocationID', 'DOLocationID']
clean_jan_data[categorical] = clean_jan_data[categorical].astype(str)
train_dicts = clean_jan_data[categorical].to_dict(orient='records')

hwOne_dv = DictVectorizer()
X_train = hwOne_dv.fit_transform(train_dicts)
print(X_train.shape)

(3009173, 515)


In [8]:
# dimensionality of feature matrix
print(f"The dimensionality of the matrix is {X_train.shape[1]}")

The dimensionality of the matrix is 515


<h3>Q5. Training the model</h3>

In [9]:
# train linear regression
target = 'duration'
y_train = clean_jan_data[target].values

hwOne_lin_reg = LinearRegression()
hwOne_lin_reg.fit(X_train, y_train)
y_pred = hwOne_lin_reg.predict(X_train)
mse = mean_squared_error(y_train, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error on train set is: {rmse}")

Root Mean Squared Error on train set is: 7.6492610279057605


<h3>Q6. Evaluating the model</h3>

In [10]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

In [11]:
# get validation dataset
#clean_feb_data = read_dataframe('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
categorical = ['PULocationID', 'DOLocationID']
feb_data['duration'] = feb_data.tpep_dropoff_datetime - feb_data.tpep_pickup_datetime
feb_data.duration = feb_data.duration.apply(lambda td: td.total_seconds() / 60)
clean_feb_data = feb_data[(feb_data.duration >= 1) & (feb_data.duration <= 60)]
clean_feb_data[categorical] = clean_feb_data[categorical].astype(str)
clean_feb_data.shape

(2855951, 20)

In [1]:
# evaluate the model
val_dicts = clean_feb_data[categorical].to_dict(orient='records')
X_val = hwOne_dv.transform(val_dicts)
y_val = clean_feb_data[target].values

hwOne_lin_reg = LinearRegression()
hwOne_lin_reg.fit(X_train, y_train)
y_pred = hwOne_lin_reg.predict(X_val)
mse = mean_squared_error(y_val, y_pred)
rmse = np.sqrt(mse)
print(f"Root Mean Squared Error on validation set is: {rmse}")

NameError: name 'clean_feb_data' is not defined

In [ ]:
with open('models/hwOne_lin_reg.bin', 'wb') as f_out:
    pickle.dump((hwOne_dv, hwOne_lin_reg), f_out)